##### Copyright 2024 Google LLC.

# Explore vision capabilities with the Gemini API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/vision"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

The Gemini API is able to process images and videos, enabling a multitude of
 exciting developer use cases. Some of Gemini's vision capabilities include
 the ability to:

*   Caption and answer questions about images
*   Transcribe and reason over PDFs, including long documents up to 2 million token context window
*   Describe, segment, and extract information from videos,
including both visual frames and audio, up to 90 minutes long
*   Detect objects in an image and return bounding box coordinates for them

This tutorial demonstrates some possible ways to prompt the Gemini API with
images and video input, provides code examples,
and outlines prompting best practices with multimodal vision capabilities.
All output is text-only.

## Setup

Before you use the File API, you need to install the Gemini API SDK package and configure an API key. This section describes how to complete these setup steps.

### Install the Python SDK and import packages

The Python SDK for the Gemini API is contained in the [google-generativeai](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip.

In [6]:
!pip install -q -U google-generativeai
!pip install -U -q "google-generativeai>=0.7.2"


Import the necessary packages.

In [2]:
import google.generativeai as genai
from IPython.display import Markdown

### Set up your API key

The File API uses API keys for authentication and access. Uploaded files are associated with the project linked to the API key. Unlike other Gemini APIs that use API keys, your API key also grants access to data you've uploaded to the File API, so take extra care in keeping your API key secure. For more on keeping your keys
secure, see [Best practices for using API
keys](https://support.google.com/googleapi/answer/6310037).

Store your API key in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or are unfamiliar with Colab Secrets, refer to the [Authentication quickstart](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb).

In [8]:
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Prompting with images

In this tutorial, you will upload images using the File API or as inline data and generate content based on those images.

### Technical details (images)
Gemini 1.5 Pro and Flash support a maximum of 3,600 image files.

Images must be in one of the following image data [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):

-   PNG - `image/png`
-   JPEG - `image/jpeg`
-   WEBP - `image/webp`
-   HEIC - `image/heic`
-   HEIF - `image/heif`

Each image is equivalent to 258 tokens.

While there are no specific limits to the number of pixels in an image besides the model’s context window, larger images are scaled down to a maximum resolution of 3072x3072 while preserving their original aspect ratio, while smaller images are scaled up to 768x768 pixels. There is no cost reduction for images at lower sizes, other than bandwidth, or performance improvement for images at higher resolution.

For best results:

*   Rotate images to the correct orientation before uploading.
*   Avoid blurry images.
*   If using a single image, place the text prompt after the image.

## Image input

For total image payload size less than 20MB, it's recommended to either upload
base64 encoded images or directly upload locally stored image files.

### Base64 encoded images

You can upload public image URLs by encoding them as Base64 payloads.
You can use the httpx library to fetch the image URLs.
The following code example shows how to do this:

In [9]:
import httpx
import base64

# Retrieve an image
image_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image = httpx.get(image_path)

# Choose a Gemini model
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Create a prompt
prompt = "Caption this image."
response = model.generate_content(
    [
        {
            "mime_type": "image/jpeg",
            "data": base64.b64encode(image.content).decode("utf-8"),
        },
        prompt,
    ]
)

Markdown(">" + response.text)

>A view of London, England showcases the Palace of Westminster (Houses of Parliament) and Big Ben with the London Eye in the background under a cloudy sky. Other landmarks and buildings are also visible throughout the cityscape.

### Multiple images

To prompt with multiple images in Base64 encoded format, you can do the
following:

In [10]:
import httpx
import base64

# Retrieve two images
image_path_1 = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image_path_2 = "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"

image_1 = httpx.get(image_path_1)
image_2 = httpx.get(image_path_2)

# Create a prompt
prompt = "Generate a list of all the objects contained in both images."

response = model.generate_content([
{'mime_type':'image/jpeg', 'data': base64.b64encode(image_1.content).decode('utf-8')},
{'mime_type':'image/jpeg', 'data': base64.b64encode(image_2.content).decode('utf-8')}, prompt])

Markdown(response.text)

**Image 1: London cityscape**

* London Eye (ferris wheel)
* Houses of Parliament (including Big Ben)
* The Shard
* River Thames
* Various other buildings, including modern skyscrapers and historical architecture
* Cranes (indicating construction)
* Trees

**Image 2: Jetpack Backpack Sketch**

* Backpack shape
* Padded strap support
* USB-C charging port
* Retractable boosters with flames/steam
* Annotations indicating:
    * Fits 18" laptop
    * Lightweight/looks like a normal backpack
    * 15-minute battery life
    * Steam-powered/green/clean propulsion

### Upload one or more locally stored image files

Alternatively, you can upload one or more locally stored image files..

You can download and use our drawings of [piranha-infested waters](https://storage.googleapis.com/generativeai-downloads/images/piranha.jpg) and a [firefighter with a cat](https://storage.googleapis.com/generativeai-downloads/images/firefighter.jpg). First, save these files to your local directory.

Then click **Files** on the left sidebar. For each file, click the **Upload** button, then navigate to that file's location and upload it:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files. Smaller files can instead be called locally from the Gemini API:


In [11]:
import PIL.Image

sample_file_2 = PIL.Image.open('/content/6237845806350714384.jpg')
sample_file_3 = PIL.Image.open('/content/6237845806350714385.jpg')

In [12]:
import google.generativeai as genai

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Create a prompt.
prompt = "Write an advertising jingle based on the items in both images."

response = model.generate_content([sample_file_2, sample_file_3, prompt])

Markdown(response.text)

(Upbeat, slightly quirky music)

Stuck on a cliff, shark in the way?
Home's just a hop, but what do you say?
Need a rescue, a bridge, or a boat?
We've got solutions, don't lose all hope!

(Music changes, becomes more mellow)

Kitty in a tree, can't get her down?
Fireman Fred's in town!
Ladders and nets, we're here to save the day,
From tricky cliffs to branches that sway!

(Music becomes faster, combines both styles)

So whatever your problem, big or small,
Give us a call, we'll handle it all!
Cliffside jumps or feline feats,
We've got the answers, from your head to your feet!
(Sound of a splash and a meow) 
Problem Solvers, here for YOU!


Note that these inline data calls don't include many of the features available
through the File API, such as getting file metadata,
[listing](https://ai.google.dev/gemini-api/docs/vision?lang=python#list-files),
or [deleting files](https://ai.google.dev/gemini-api/docs/vision?lang=python#delete-files).

### Large image payloads

#### Upload an image file using the File API

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files.

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but cannot be downloaded from the API. It is available at no cost in all regions where the Gemini API is available.

Upload the image using [`media.upload`](https://ai.google.dev/api/rest/v1beta/media/upload) and print the URI, which is used as a reference in Gemini API calls.

In [13]:
!curl -o jetpack.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2461k      0 --:--:-- --:--:-- --:--:-- 2476k


In [14]:
# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="jetpack.jpg",
                            display_name="Jetpack drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/5py47159xjq5


The `response` shows that the File API stored the specified `display_name` for the uploaded file and a `uri` to reference the file in Gemini API calls. Use `response` to track how uploaded files are mapped to URIs.

Depending on your use case, you can also store the URIs in structures such as a `dict` or a database.

#### Verify image file upload and get metadata

You can verify the API successfully stored the uploaded file and get its metadata by calling [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) through the SDK. Only the `name` (and by extension, the `uri`) are unique. Use `display_name` to identify files only if you manage uniqueness yourself.

In [15]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/5py47159xjq5


Depending on your use case, you can store the URIs in structures, such as a `dict` or a database.

#### Prompt with the uploaded image and text

After uploading the file, you can make GenerateContent requests that reference the File API URI. Select the generative model and provide it with a text prompt and the uploaded image.

In [16]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Describe how this product might be manufactured."])

Markdown(response.text)

Manufacturing a jetpack backpack, even a conceptual steam-powered one, would involve a complex process integrating several different manufacturing techniques:

1. **Backpack Body/Frame:**  Likely injection-molded plastic for the main body to keep it lightweight and cost-effective.  Reinforced sections (perhaps carbon fiber or other strong, lightweight composite materials) would be integrated around the booster attachment points, inside the straps, and in other high-stress areas.  This might involve processes like compression molding or resin transfer molding. Sewing would be used for assembling fabric components and attaching them to the molded plastic components.

2. **Boosters (Retractable):**  The retractable mechanism would likely involve precision machining of small metal parts for gears, hinges, and locking mechanisms.  The booster nozzles themselves could be 3D printed from high-temperature resistant metal alloys like Inconel or titanium, or formed using metal injection molding.

3. **Steam Generation System:** This is the most complex part and would involve several sub-assemblies:

    * **Boiler/Heat Exchanger:** This could be a small, high-pressure vessel made from a strong, heat-resistant material, potentially a titanium alloy. Manufacturing would likely involve advanced welding techniques and pressure testing.
    * **Water Reservoir:**  A flexible, sealed bladder or a rigid tank made from a lightweight, chemically resistant plastic would hold the water.  This could be manufactured using blow molding or rotational molding.
    * **Valves and Piping:**  Precision machining would create small, high-pressure valves to control steam flow.  Piping might be thin-walled stainless steel or titanium tubing.
    * **Ignition/Control System:**  Electronic components would manage the heating element, monitor pressure and temperature, and control the booster activation. This involves standard PCB manufacturing and electronics assembly.

4. **Power Source/Battery:**  Lithium-ion battery packs would be assembled using existing battery cell manufacturing technologies and integrated into a custom battery management system within the backpack.

5. **USB-C Charging Port:** Standard electronic components and connectors would be integrated into the backpack during assembly.

6. **Padding and Straps:**  Standard textile manufacturing techniques would create the padded straps and back panel, likely using materials like nylon webbing, foam padding, and breathable fabrics.

7. **Assembly:**  The final assembly process would be complex and would involve multiple steps:
    * Attaching the boosters and their retraction mechanism to the backpack frame.
    * Integrating the steam generation system and securely mounting it within the backpack.
    * Connecting the battery, control system, and charging port.
    * Adding the padding, straps, and other finishing touches.
    * Rigorous quality control and testing to ensure safety and functionality.


This describes a potential manufacturing process.  The actual methods would depend on design specifics, material choices, and cost considerations. It's worth noting that steam-powered flight in a backpack form factor presents significant engineering challenges, particularly regarding safety and power efficiency. This is very much a conceptual idea.


## Capabilties

This section outlines specific vision capabilities of the Gemini model, including object detection and bounding box coordinates.

### Get bounding boxes

Gemini models are trained to return bounding box coordinates as relative widths or heights in the range of [0, 1]. These values are then scaled by 1000 and converted to integers. Effectively, the coordinates represent the bounding box on a 1000x1000 pixel version of the image. Therefore, you'll need to convert these coordinates back to the dimensions of your original image to accurately map the bounding boxes.

In [17]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Create a prompt to detect bounding boxes.
prompt = "Return a bounding box for each of the objects in this image in [ymin, xmin, ymax, xmax] format."
response = model.generate_content([sample_file_2, prompt])

Markdown(response.text)

- 361 248 620 326 stick figure
- 613 497 704 574 fish
- 458 734 629 895 home sign


The model returns bounding box coordinates in the format
`[ymin, xmin, ymax, xmax]`. To convert these normalized coordinates
to the pixel coordinates of your original image, follow these steps:

1.    Divide each output coordinate by 1000.
1.    Multiply the x-coordinates by the original image width.
1.    Multiply the y-coordinates by the original image height.

To explore more detailed examples of generating bounding box coordinates and
visualizing them on images, review our [Object Detection cookbook example](https://github.com/google-gemini/cookbook/blob/main/examples/Object_detection.ipynb).

## Prompting with video

In this tutorial, you will upload a video using the File API and generate content based on those images.

### Technical details (video)

Gemini 1.5 Pro and Flash support up to approximately an hour of video data.

Video must be in one of the following video format [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
  -   `video/mp4`
  -   `video/mpeg`
  -   `video/mov`
  -   `video/avi`
  -   `video/x-flv`
  -   `video/mpg`
  -   `video/webm`
  -   `video/wmv`
  -   `video/3gpp`

The File API service currently extracts image frames from videos at 1 frame per second (FPS) and audio at 1Kbps, single channel, adding timestamps every second. These rates are subject to change in the future for improvements in inference.

**NOTE:** The finer details of fast action sequences may be lost at the 1 FPS frame sampling rate. Consider slowing down high-speed clips for improved inference quality.

Individual frames are 258 tokens, and audio is 32 tokens per second. With metadata, each second of video becomes ~300 tokens, which means a 1M context window can fit slightly less than an hour of video.

To ask questions about time-stamped locations, use the format `MM:SS`, where the first two digits represent minutes and the last two digits represent seconds.

For best results:

*   Use one video per prompt.
*   If using a single video, place the text prompt after the video.

### Upload a video file to the File API

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but they cannot be downloaded using any API. It is available at no cost in all regions where the Gemini API is available.

The File API accepts video file formats directly. This example uses the short NASA film ["Jupiter's Great Red Spot Shrinks and Grows"](https://www.youtube.com/watch?v=JDi4IdtvDVE0). Credit: Goddard Space Flight Center (GSFC)/David Ladd (2018).

> "Jupiter's Great Red Spot Shrinks and Grows" is in the public domain and does not show identifiable people. ([NASA image and media usage guidelines.](https://www.nasa.gov/nasa-brand-center/images-and-media/))

Start by retrieving the short video:

In [18]:
!wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4

--2025-03-13 05:34:02--  https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.101.207, 142.250.141.207, 142.251.2.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.101.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238090979 (227M) [video/mp4]
Saving to: ‘GreatRedSpot.mp4’

GreatRedSpot.mp4    100%[===================>] 227.06M   111MB/s    in 2.0s    

2025-03-13 05:34:05 (111 MB/s) - ‘GreatRedSpot.mp4’ saved [238090979/238090979]



Upload the video to the File API and print the URI.

In [19]:
video_file_name = "GreatRedSpot.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/h0c93a2ir457


### Verify file upload and check state

Verify the API has successfully received the files by calling the [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) method.

**NOTE**: Video files have a `State` field in the File API. When a video is uploaded, it will be in the `PROCESSING` state until it is ready for inference. Only `ACTIVE` files can be used for model inference.

In [20]:
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

..

### Prompt with a video and text

Once the uploaded video is in the `ACTIVE` state, you can make `GenerateContent` requests that specify the File API URI for that video. Select the generative model and provide it with the uploaded video and a text prompt.

In [21]:
# Create the prompt.
prompt = "Summarize this video. Then create a quiz with answer key based on the information in the video."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})

# Print the response, rendering any Markdown
Markdown(response.text)

Making LLM inference request...


Here's a summary of the video and a quiz with an answer key:

Summary:

Jupiter is the biggest and oldest planet in the solar system. Although made of the same elements as a star, it never gained enough mass to ignite. The Great Red Spot is a giant storm on Jupiter that has been active for more than a century. However, scientists have observed that the storm is shrinking and getting rounder over time. While some believed that the shrinking would cause the wind speed in the Great Red Spot to increase, based on the physics involved when an ice skater spins faster by pulling in their arms, that theory was disproven. NASA scientists have discovered that the storm is getting taller as it shrinks, similar to what happens when a wide lump of clay is spun on a potter's wheel. Data used to make these discoveries came from several NASA missions, including Voyager, Hubble, and Juno. 

Quiz:

1. What are two superlative facts about Jupiter?
2. What is the name of the giant storm on Jupiter?
3. How long has the Great Red Spot been active?
4. What happens to the shape of the Great Red Spot as it shrinks?
5. What has been happening to the size of the Great Red Spot?
6. What did some scientists expect would happen to the wind speeds in the Great Red Spot as it shrinks?
7. What has actually happened to the Great Red Spot as it shrinks?
8. What analogy is used to explain the changes in the Great Red Spot?
9. How many Earths could fit inside the Great Red Spot before it began shrinking?
10. Which three NASA missions were used to gather data about the Great Red Spot?

Answer Key:

1. It's the largest and oldest planet in the solar system.
2. The Great Red Spot.
3. More than a century.
4. It gets rounder.
5. It is getting smaller.
6. They expected that the wind speeds would increase.
7. It has gotten taller.
8. A potter spinning a lump of clay on a wheel, making it taller as it spins.
9. Three.
10. Voyager, Hubble, and Juno.

### Refer to timestamps in the content

You can use timestamps of the form `MM:SS` to refer to specific moments in the video.

In [ ]:
# Create the prompt.
prompt = "What are the examples given at 01:05 and 01:19 supposed to show us?"

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
Markdown(response.text)

### Transcribe video and provide visual descriptions

The Gemini models can transcribe and provide visual descriptions of video content
by processing both the audio track and visual frames.
For visual descriptions, the model samples the video at a rate of **1 frame
per second**. This sampling rate may affect the level of detail in the
descriptions, particularly for videos with rapidly changing visuals.

In [ ]:
# Create the prompt.
prompt = "Transcribe the audio from this video, giving timestamps for salient events in the video. Also provide visual descriptions."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})
Markdown(response.text)

## List files

You can list all uploaded files and their URIs using `files.list_files()`.

In [ ]:
# List all files
for file in genai.list_files():
    print(f"{file.display_name}, URI: {file.uri}")

## Delete files

Files are automatically deleted after 2 days. You can also manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')